# Primer modelo usando SKLearn

## Importamos las librerías

In [36]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

## Importamos los datos con pandas

In [37]:
datos_titanic = pd.read_csv('./titanic_train.csv')

In [38]:
# Cargando los datos
entrenamiento, pruebas = train_test_split(datos_titanic,test_size=0.3)


In [39]:
entrenamiento.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,623.000000,623.000000,623.000000,499.000000,623.000000,623.000000,623.000000
mean,439.401284,0.383628,2.292135,30.661483,0.449438,0.382022,32.014104
std,258.716737,0.486660,0.838034,15.067599,0.903866,0.808992,49.643024
min,1.000000,0.000000,1.000000,0.750000,0.000000,0.000000,0.000000
25%,213.500000,0.000000,2.000000,21.000000,0.000000,0.000000,7.895800
50%,438.000000,0.000000,3.000000,29.000000,0.000000,0.000000,14.454200
75%,660.500000,1.000000,3.000000,40.000000,1.000000,0.000000,31.000000
max,890.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [40]:
entrenamiento.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
353,354,0,3,"Arnold-Franchi, Mr. Josef",male,25.0,1,0,349237,17.8000,NaN,S
449,450,1,1,"Peuchen, Major. Arthur Godfrey",male,52.0,0,0,113786,30.5000,C104,S
317,318,0,2,"Moraweck, Dr. Ernest",male,54.0,0,0,29011,14.0000,NaN,S
684,685,0,2,"Brown, Mr. Thomas William Solomon",male,60.0,1,1,29750,39.0000,NaN,S
629,630,0,3,"O'Connell, Mr. Patrick D",male,NaN,0,0,334912,7.7333,NaN,Q


## Hacemos una "limpieza" de nuestro datos antes de hacer el modelo

In [41]:
combine = [entrenamiento, pruebas]

In [42]:
# Convert string values 'male' and 'female' to int values
sex_mapping = {'male': 0, 'female': 1}
entrenamiento['Sex'] = entrenamiento['Sex'].map(sex_mapping)
pruebas['Sex'] = pruebas['Sex'].map(sex_mapping)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [43]:
calculo_edades = np.zeros((2,3))

In [44]:
for dataset in combine:
    for sex in range(0, 2):
        for pclass in range(0, 3):
            guess_df = dataset[(dataset['Sex'] == sex) & (dataset['Pclass'] == pclass+1)]['Age'].dropna()
            age_guess = guess_df.median()
            calculo_edades[sex, pclass] = int(age_guess/0.5 + 0.5) * 0.5
    
    for sex in range(0, 2):
        for pclass in range(0, 3):
            dataset.loc[(dataset.Age.isnull()) & (dataset.Sex == sex) &(dataset.Pclass == pclass+1),'Age'] = calculo_edades[sex, pclass]

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [45]:
entrenamiento = entrenamiento.drop(['Ticket', 'Cabin', 'Name', 'PassengerId', 'SibSp', 'Parch', 'Embarked'], axis=1)
pruebas = pruebas.drop(['Ticket', 'Cabin', 'Name', 'SibSp', 'Parch', 'Embarked'], axis=1)

X_train = entrenamiento.drop('Survived', axis=1)
Y_train = entrenamiento['Survived']
X_test  = pruebas.drop(["PassengerId","Survived"], axis=1)

## Creamos y entrenamos nuestro modelo

In [46]:
decision_tree = DecisionTreeClassifier()


In [47]:
decision_tree.fit(X_train, Y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

## Predecimos con nuestro árbol y la tasa de efectividad

In [48]:
Y_pred = decision_tree.predict(X_test)

In [49]:
decision_tree.score(X_train, Y_train)

0.9791332263242376